# Note
1. Run the first part (Preprocessing df with no features) of the notebook and comment all other parts
2. Run the second part (Preprocessing df_part_x.csv files on training and testing) of the notebook and comment all other parts
3. Run the third part of the notebook (Read the filtered... to do the trraining...) of the notebook and comment all other parts $\rightarrow$ The third part will give you the final results. 

# Imports

In [1]:
import os
import ipdb
import argparse
import numpy as np
import pandas as pd
import numpy.random as random
import networkx as nx
from progressbar import ProgressBar
import matplotlib.pyplot as plt
import gzip
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import csv
from joblib import dump, load
import json
pd.options.display.max_columns = 9999999

# Preprocessing df with no features ✅
Chopping the t,u,v, is_fraud and subsequently the one with all the feaatures into overlapping windows

# Preprocessing df_part_x.csv files on training and testing ✅

# Read in the `filtered...` to do the training and `to_test...` to do the testing for each window

In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import os

# Define the function for training and testing
def train_and_test(train_file, test_file, part):
    print(f"Processing part {part}...")

    # Read the training DataFrame from the file
    df_train = pd.read_csv(train_file, sep=' ')
    
    # Drop unnecessary columns
    df_train = df_train.drop(["t", "u", "v", "history_type", "history_size", "bip", "cost"], axis="columns")
    
    # Ensure no NaN values
    df_train = df_train.dropna()
    
    # Count the values of the 'is_fraud' column
    print(df_train["is_fraud"].value_counts())
    
    # Training
    X_train = df_train.drop('is_fraud', axis=1)
    y_train = df_train['is_fraud']
    
    # Create a RandomForestClassifier
    rf_classifier = RandomForestClassifier(random_state=42, n_jobs=-1)
    
    # Train the classifier on the training data
    print("Training now")
    rf_classifier.fit(X_train, y_train)

    # Testing
    print("Testing now")
    chunk_size = 1000000
    roc_auc_scores = []

    for chunk in pd.read_csv(test_file, sep=' ', chunksize=chunk_size):
        try:
            chunk = chunk.dropna()
            chunk = chunk.drop(["t", "u", "v", "history_type", "history_size", "bip", "cost"], axis="columns")
            X_chunk = chunk.drop('is_fraud', axis=1)
            y_chunk = chunk['is_fraud']
            y_pred_chunk = rf_classifier.predict(X_chunk)

            roc_auc_chunk = roc_auc_score(y_chunk, rf_classifier.predict_proba(X_chunk)[:, 1])

            roc_auc_scores.append(roc_auc_chunk)

            print(f"roc_auc_chunk: {roc_auc_chunk:.4f}")
            print("--------------------------------------------------------------")
            
        except Exception as e:
            print(f"Error processing chunk: {e}")
            continue

    if roc_auc_scores:
        avg_roc_auc = sum(roc_auc_scores) / len(roc_auc_scores)

        print(f"Average ROC-AUC Score for df part {part}: {avg_roc_auc:.4f}")
        return avg_roc_auc
    else:
        print(f"No valid chunks processed for part {part}.")
        return None

In [3]:
avg_roc_auc_list = []

base_dir = 'TrainingTestingFiles'

for i in range(1, 51):
    train_file = os.path.join(base_dir, f'train_df_no_features_undersampled_df_part_df_part_{i}.txt')
    test_file = os.path.join(base_dir, f'test_df_no_features_r0.7_df_part_df_part_{i}.txt')
    avg_roc_auc = train_and_test(train_file, test_file, i)
    avg_roc_auc_list.append(avg_roc_auc)

print(f"Overall Average ROC-AUC Score: {sum(avg_roc_auc_list) / len(avg_roc_auc_list):.4f}")

Processing part 1...
0    95392
1    95367
Name: is_fraud, dtype: int64
Training now
Testing now
roc_auc_chunk: 0.9187
--------------------------------------------------------------
Average ROC-AUC Score for df part 1: 0.9187
Processing part 2...
1    109301
0    109301
Name: is_fraud, dtype: int64
Training now
Testing now
roc_auc_chunk: 0.9168
--------------------------------------------------------------
Average ROC-AUC Score for df part 2: 0.9168
Processing part 3...
0    72177
1    72177
Name: is_fraud, dtype: int64
Training now
Testing now
roc_auc_chunk: 0.9462
--------------------------------------------------------------
Average ROC-AUC Score for df part 3: 0.9462
Processing part 4...
1    111458
0    111458
Name: is_fraud, dtype: int64
Training now
Testing now
roc_auc_chunk: 0.9238
--------------------------------------------------------------
Average ROC-AUC Score for df part 4: 0.9238
Processing part 5...
1    132609
0    132609
Name: is_fraud, dtype: int64
Training now
Testi

0    59818
1    59818
Name: is_fraud, dtype: int64
Training now
Testing now
roc_auc_chunk: 0.9389
--------------------------------------------------------------
Average ROC-AUC Score for df part 37: 0.9389
Processing part 38...
0    62391
1    62391
Name: is_fraud, dtype: int64
Training now
Testing now
roc_auc_chunk: 0.9373
--------------------------------------------------------------
Average ROC-AUC Score for df part 38: 0.9373
Processing part 39...
0    64220
1    64220
Name: is_fraud, dtype: int64
Training now
Testing now
roc_auc_chunk: 0.9374
--------------------------------------------------------------
Average ROC-AUC Score for df part 39: 0.9374
Processing part 40...
1    71715
0    71715
Name: is_fraud, dtype: int64
Training now
Testing now
roc_auc_chunk: 0.9301
--------------------------------------------------------------
Average ROC-AUC Score for df part 40: 0.9301
Processing part 41...
0    76696
1    76696
Name: is_fraud, dtype: int64
Training now
Testing now
roc_auc_chun

In [4]:
if avg_roc_auc_list:
    lowest_roc_auc = min(avg_roc_auc_list)
    highest_roc_auc = max(avg_roc_auc_list)
    avg_roc_auc = sum(avg_roc_auc_list) / len(avg_roc_auc_list)

    print("Lowest ROC-AUC:", lowest_roc_auc)
    print("Highest ROC-AUC:", highest_roc_auc)
    print("Average ROC-AUC:", avg_roc_auc)

else:
    print("No valid data for calculating statistics.")


Lowest ROC-AUC: 0.9037455225191943
Highest ROC-AUC: 0.9607985467208028
Average ROC-AUC: 0.9336851853070124


In [5]:
avg_roc_auc_list

[0.9186645902329735,
 0.9167812274032962,
 0.9462228677892658,
 0.9237517823500113,
 0.9174041638053452,
 0.9089669162417493,
 0.9171048336306618,
 0.9284328504510404,
 0.9083000517483193,
 0.911961881851153,
 0.9037455225191943,
 0.9207918920138527,
 0.9339058535433584,
 0.9378631539234141,
 0.942015262602313,
 0.9438789875904707,
 0.9421472497377066,
 0.9491268434851483,
 0.952040784058464,
 0.9420218972505038,
 0.9523943680898806,
 0.9522943976190679,
 0.9536295495266506,
 0.9541392397753244,
 0.9536466347159722,
 0.9527771709675913,
 0.9507787847882284,
 0.949259305800656,
 0.9496071431402917,
 0.9607985467208028,
 0.9507934274577184,
 0.9499268977437265,
 0.9483040805398758,
 0.9449130179319263,
 0.9404676017298202,
 0.9404459402222656,
 0.9388757306361764,
 0.9373248176058294,
 0.9374437981794572,
 0.9300591920899626,
 0.9268265558986473,
 0.9229110513809137,
 0.920772594931503,
 0.9162038354978764,
 0.9150712382475794,
 0.9169780756678063,
 0.9172413528884575,
 0.912713599644154

In [6]:
len(avg_roc_auc_list)

50

In [7]:
df = pd.DataFrame(avg_roc_auc_list, columns=["Avg ROC AUC"])

df.to_excel("avg_roc_auc_list.xlsx", index=False)